# Collectl Dsk Log Analysis

## Functionalities
- Plot disk I/O utilization graphs.

## Input
Log files are read from a directory in `../data`. This directory is assumed to have the following structure:
```
logs/
  [node-1]/
    collectl.tar.gz
  ...
  [node-n]/
    collectl.tar.gz
```

## Notebook Configuration

In [ ]:
########## GENERAL
# Name of the directory in `../data`
EXPERIMENT_DIRNAME = "BuzzBlogBenchmark_[TIMESTAMP]"

########## DISK I/O
# Analyzed metric (options: "name", "reads", "rmerge", "rkbytes", "waitr", "writes", "wmerge", "wkbytes", "waitw",
# "request", "quelen", "wait", "svctim", "util")
COLLECTL_DSK_METRIC = "writes"

## Notebook Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
import sys
import warnings
warnings.filterwarnings("ignore")

sys.path.append(os.path.abspath(os.path.join("..")))
from utils.utils import *

experiment_dirpath = os.path.join(os.path.abspath(""), "..", "data", EXPERIMENT_DIRNAME)

## Log Parsing & Processing

In [ ]:
# Build data frame
dsk = pd.concat([df[2] for df in get_collectl_dsk_df(experiment_dirpath)])

In [ ]:
# Extract experiment information
start_time = get_experiment_start_time(experiment_dirpath)
node_names = get_node_names(experiment_dirpath)

In [ ]:
# (Re) Build columns
dsk["timestamp"] = dsk.apply(lambda r: (r["timestamp"] - start_time).total_seconds(), axis=1)
dsk["window_1000"] = dsk["timestamp"].round(0).multiply(1000)

In [ ]:
# (Re) Create index
dsk.set_index("timestamp", inplace=True)
dsk.sort_index(inplace=True)

## Disk Monitoring

In [ ]:
fig = plt.figure(figsize=(24, len(node_names) * 12))
for (i, node_name) in enumerate(sorted(node_names)):
    # Data frame
    df = dsk[(dsk["node_name"] == node_name)].groupby(["window_1000"])[COLLECTL_DSK_METRIC].mean()
    # Plot
    ax = fig.add_subplot(len(node_names), 1, i + 1)
    ax.set_xlim((0, df.index.max()))
    ax.grid(alpha=0.75)
    df.plot(ax=ax, kind="line", title="%s - Disk Utilization" % node_name, xlabel="Time (millisec)", ylabel="%s" % COLLECTL_DSK_METRIC, grid=True)

In [ ]:
########## ZOOM IN
# Minimum time (in sec)
MIN_TIME = None
# Maximum time (in sec)
MAX_TIME = None

if MIN_TIME and MAX_TIME:
    fig = plt.figure(figsize=(24, len(node_names) * 12))
    for (i, node_name) in enumerate(node_names):
        # Data frame
        df = dsk[(dsk["node_name"] == node_name) & (dsk.index >= MIN_TIME) & (dsk.index <= MAX_TIME)].groupby(["timestamp", "hw_no"])[COLLECTL_DSK_METRIC].mean().unstack()
        # Plot
        ax = fig.add_subplot(len(node_names), 1, i + 1)
        ax.set_xlim((df.index.min(), df.index.max()))
        ax.grid(alpha=0.75)
        df.plot(ax=ax, kind="line", title="%s - Disk Utilization" % node_name, xlabel="Time (millisec)", ylabel="%s" % COLLECTL_DSK_METRIC, grid=True, legend=True)